# PSG Complete Data Analysis - FootballDecoded

### Objetivo del Análisis

Este notebook realiza una **extracción exhaustiva y organizada** de todos los datos disponibles del **Paris Saint-Germain** para las temporadas **2023/24 y 2024/25**, utilizando el conjunto completo de wrappers desarrollados en FootballDecoded.

**Datos a extraer:**
- Estadísticas de equipo (temporada completa por competición)
- Estadísticas individuales (todos los jugadores de la plantilla)
- Eventos de partidos (goles, pases, disparos, etc.)
- Datos espaciales (coordenadas, mapas de calor, redes de pase)
- Métricas avanzadas (xG, xA, PPDA, build-up chains)
- Análisis específico del partido PSG vs Auxerre (01/09/2024)

**Fuentes de datos:**
- **FBref**: Estadísticas completas y eventos
- **Understat**: Métricas avanzadas (xGChain, PPDA, etc.)
- **WhoScored**: Datos espaciales y coordenadas

In [52]:
# Dependencias base
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Union
from datetime import datetime
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

import sys
import os
    
# Ir al directorio correcto donde está instalado el paquete
data_dir = "/home/oriol/FD/Data"
sys.path.insert(0, data_dir)
    
from scrappers import FBref, Understat, WhoScored
from wrappers import *